In [ ]:
import math
import random
from math import radians

f0 = """D:\iterations\\0_{}.xml"""

# Dynamic viscosity of water
# source http://www.viscopedia.com/viscosity-tables/substances/water/
water_viscocities = {20:0.0010016, 21:0.0009775, 22:0.0009544,23:0.0009321, 24:0.0009107,25:0.00089}

tempC = 20
dt = 0.0025

if tempC >= 20 and tempC <= 25:
    const_Boltzmann = 1.3806504e-23
    # Temperature (Kelvins) (20-25C) + 273.15
    const_Temperature_K = tempC + 273.15
    const_water_viscosity = water_viscosities[tempC - 20]
    print("t",tempC,"tK", const_Temperature_K, "visc",const_water_viscosity)
    print("const_Boltzmann * const_Temperature_K", const_Boltzmann * const_Temperature_K)
    print("6. * math.pi * const_water_viscosity", 6. * math.pi * const_water_viscosity)
else:
    print("Error")

t 20 tK 293.15 visc 0.0010016
const_Boltzmann * const_Temperature_K 4.0473766476e-21
6. * math.pi * const_water_viscosity 0.018879715211013223


In [16]:
class EV:
    def random_coord(self):
        return random.randint(-90, 90)
    
    def random_diameter_nm(self):
        return (random.randint(3,30) * 10)
        
    def __init__(self, *args, **kwargs):
        self._id = kwargs.get('_id', 0)
        self.x_1 = kwargs.get('x_1', self.random_coord())
        self.y_1 = kwargs.get('y_1', self.random_coord())
        self.vx = kwargs.get('vx', 0)
        self.vy = kwargs.get('vy', 0)
        self.diameter_um = kwargs.get('diameter_nm', self.random_diameter_nm()) /1000
        
        self.radius_um = (self.diameter_um / 2)
        self.radius_m = self.radius_um * 0.000001
        self.mass_Kg = (5.47991E-20 / 100) * self.diameter_um

        # Compute the radius-dependent diffusion coefficient defined as
        # D = (boltzmann tempK)/(6 pi fluidViscosity particleRadius)
        self.diffusion_rate_m = ((const_Boltzmann * const_Temperature_K) 
                                 / (6. * math.pi * const_water_viscosity * self.radius_m))
        self.diffusion_rate_um = self.diffusion_rate_m * 1000000000000.
        
        # compute the RMS displacement per second which is diffusion-rate dependat
        self.rms_displacement_um = math.sqrt(2 * self.diffusion_rate_um)
        self.rms_displacement_m = math.sqrt(2 * self.diffusion_rate_m)
    
    def random_initial_impulse(self):
        init_angle = random.randrange(0,359,1)
        self.vx = self.rms_displacement_um * math.cos(radians(init_angle))
        self.vy = self.rms_displacement_um * math.sin(radians(init_angle))
        self.x = self.x_1 + self.vx * dt
        self.y = self.y_1 + self.vy * dt
    
    def diffuse_h(self, positive=True):
        self.vx = self.rms_displacement_um
        if not positive:
            self.vx *= -1
        self.x = self.x_1 + self.vx * dt
        self.y = 0
    
    def diffuse_v(self, positive=True):
        self.vy = self.rms_displacement_um
        if not positive:
            self.vy *= -1
        self.y = self.y_1 + self.vy * dt
        self.x = 0
    
    def getXmlDescription(self):
        return """
    <xagent>
        <name>EV</name>
        <id>{:d}</id>
        <x>{:f}</x>
        <y>{:f}</y>
        <x_1>{:f}</x_1>
        <y_1>{:f}</y_1>
        <vx>{:.10f}</vx>
        <vy>{:.10f}</vy>
        <mass>{:.25f}f</mass>
        <colour/>
        <radius_um>{:f}</radius_um>
        <radius_m>{:.10f}</radius_m>
        <diameter_um>{:f}</diameter_um>        
        <diffusion_rate_m>{:.25f}</diffusion_rate_m>
        <diffusion_rate_um>{:.25f}</diffusion_rate_um>
        <rms_displacement_um>{:.25f}</rms_displacement_um>
        <rms_displacement_m>{:.25f}</rms_displacement_m>
        <closest_ev_id>-1</closest_ev_id>
        <closest_ev_distance>-1.f</closest_ev_distance>
        <closest_wall_id>-1</closest_wall_id>
        <closest_wall_distance>-1.f</closest_wall_distance>
    </xagent>""".format(self._id, self.x, self.y,self.x_1, self.y_1, self.vx, self.vy, self.mass_Kg,
                        self.radius_um, self.radius_m, self.diameter_um, 
                        self.diffusion_rate_m, self.diffusion_rate_um, self.rms_displacement_um, 
                                self.rms_displacement_m)

In [17]:
e1 = EV(x_1=0,y_1=0)
e1.random_initial_impulse()
print(e1.getXmlDescription())


    <xagent>
        <name>EV</name>
        <id>0</id>
        <x>-0.000286</x>
        <y>0.005449</y>
        <x_1>0.000000</x_1>
        <y_1>0.000000</y_1>
        <vx>-0.1142307421</vx>
        <vy>2.1796524035</vy>
        <mass>0.0000000000000000000000986f</mass>
        <colour/>
        <radius_um>0.090000</radius_um>
        <radius_m>0.0000000900</radius_m>
        <diameter_um>0.180000</diameter_um>        
        <diffusion_rate_m>0.0000000000023819666312428</diffusion_rate_m>
        <diffusion_rate_um>2.3819666312427676402307952</diffusion_rate_um>
        <rms_displacement_um>2.1826436407452169952136956</rms_displacement_um>
        <rms_displacement_m>0.0000021826436407452166066</rms_displacement_m>
        <closest_ev_id>-1</closest_ev_id>
        <closest_ev_distance>-1.f</closest_ev_distance>
        <closest_wall_id>-1</closest_wall_id>
        <closest_wall_distance>-1.f</closest_wall_distance>
    </xagent>


In [25]:
def writeXmlFromPresets(xs, ys, diameters):
    if len(xs) == len(ys) and len(ys) == len(diameters):
        with open(f0, 'w') as of:
            header = """<states>
    <itno>0</itno>
    <environment></environment>"""
            of.write(header)

            for i in range(len(xs)):
                e = EV(_id= i, x_1=xs[i], y_1=ys[i], diameter_nm=diameters[i])
                if len(xs) > 1:
                    e.diffuse_h(i < (len(xs)/2))
                else:
                    e.diffuse_h()
                of.write(e.getXmlDescription())
            of.write("""
</states>""")
    else:
        printf("list sizes do not match")

def writeXmlForManyEVs(**kwargs):
    n = kwargs.get('n', 50)
    
    with open(f0.format(n), 'w') as of:
        header = """<states>
    <itno>0</itno>
    <environment></environment>"""
        of.write(header)

        for i in range(n):
            e = EV(**dict(kwargs, _id=i))
            e.random_initial_impulse()
            of.write(e.getXmlDescription())
        of.write("""
</states>""")

In [19]:
# PRESET for 2 EVS in parallel, diffusing against each other
xs = [-5., 10.]
ys = [0, 0]
diameters_nm = [100., 50.]

writeXmlFromPresets(xs, ys, diameters_nm)

In [ ]:
# PRESET for 9 EVS at edges of screen, diffusing to the center
minX = 180
maxX = 180
minY = 100
maxY = 100
midY = 10

xs = [-minX, minX, -(int)(minX/2), (int)(minX/2), 0, -maxX, maxX, -(int)(maxX/2), (int)(maxX/2), 0]
ys = [-minY, -minY, -(int)(minY/2), -(int)(minY/2), -midY, maxY, maxY, (int)(maxY/2), (int)(maxY/2), midY]
diameters_um = [30, 30, 50, 50, 100, 100, 200, 200, 300, 300]

writeXmlFromPresets(xs, ys, diameters_um)

In [43]:
for i in range(15,21):
    print(2**i)

32768
65536
131072
262144
524288
1048576


In [44]:
for i in range(15,21):
    writeXmlForManyEVs(n=2**i,x_1=0,y_1=0)

In [46]:
writeXmlForManyEVs(n=512,x_1=0,y_1=0)